In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys, os
sys.path.append('..')

import numpy as np
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import accuracy_score
from sklearn.grid_search import GridSearchCV

from get_data import get_data, get_splitted_data, proteins, _get_raw_data
from models.active_model import ActiveLearningExperiment
from models.strategy import *
from models.utils import ObstructedY
from experiments.utils import *

from misc.config import c
from sklearn.datasets import load_svmlight_file
data_dir = c['DATA_DIR']

In [ ]:
rng = np.random.RandomState(666)

X, y = load_svmlight_file(os.path.join(data_dir, "5ht6_ExtFP.libsvm"))

y = ObstructedY(y)
y.query(rng.randint(0, X.shape[0], 100))

model = SVC(C=1, kernel='linear')
model.fit(X[y.known], y[y.known])

In [ ]:
D = pairwise_distances(X, metric=jaccard_dist)

In [ ]:
original_picked, original_score = quasi_greedy_batch(X, y, sample_first=False,
                                    current_model=model, c=1.0, rng=rng, 
                                    batch_size=20, D=D)
picked, score = multiple_pick_best(X, y, current_model=model, rng=rng, k=10, 
                                    batch_size=20, D=D, c=1.0)

print original_score, score

In [ ]:
picked

In [ ]:
from experiments.utils import get_experiment_results
import numpy as np

protein = '5ht6'
fingerprint = "ExtFP"
seed = 666
warm_start_percentage = 0.05
batch_size = 20
param_grid = {'h': [100]}

loader = ["get_splitted_data", {
    "seed": seed,
    "valid_size": 0.15,
    "n_folds": 2,
    'percent': 0.5}]

strategy = ('random_query', {})
preprocess_fncs = []

randomNB_exp = get_experiment_results("fit_randomNB_%s_%s_%s" % (strategy[0], protein, fingerprint))


In [ ]:
exp = get_best(randomNB_exp.experiments, 'auc_wac_score_concept')

In [ ]:
exp.monitors[0].keys()

In [ ]:
plot_monitors(exp, keys='times', folds='mean')

In [ ]:
for k, v in best.results.iteritems():
    if 'auc_mean_wac' in k:
        print k, v

In [ ]:
for k, v in best.results.iteritems():
    if 'auc_wac' in k:
        print k, v

In [ ]:
for protein in proteins:
    _, y = _get_raw_data(protein, "ExtFP")
    print protein, y.shape[0]

In [ ]:

loader = ["get_splitted_data_clusterwise", {
        "seed": 666,
        "valid_size": 0.15,
        "n_folds": 1}]
preprocess_fncs = [["to_binary", {"all_below": True}]]
comps = [(p, 'ExtFP') for p in proteins]
    
data = get_data(comps, loader, preprocess_fncs)

In [ ]:
for key, value in data.iteritems():
    print key
    fold = value[0][0]
    print "train data", fold['X_train']['data'].shape[0]
    print "valid data", fold['X_valid']['data'].shape[0]
    print "A", len(fold['X_train']['cluster_A'])
    print "B", len(fold['X_train']['cluster_B'])
    print 
